<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/master/Realtime_Voice_Changer_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### w-okada's Voice Changer | **Google Colab**

---

##**READ ME - VERY IMPORTANT**

This is an attempt to run [Realtime Voice Changer](https://github.com/w-okada/voice-changer) on Google Colab, still not perfect but is totally usable, you can use the following settings for better results:

If you're using a index: `f0: RMVPE_ONNX | Chunk: 112 or higher | Extra: 8192`\
If you're not using a index: `f0: RMVPE_ONNX | Chunk: 96 or higher | Extra: 16384`\
**Don't forget to select your Colab GPU in the GPU field (<b>Tesla T4</b>, for free users)*
> Seems that PTH models performance better than ONNX for now, you can still try ONNX models and see if it satisfies you


*You can always [click here](https://github.com/YunaOneeChan/Voice-Changer-Settings) to check if these settings are up-to-date*
<br><br>

---

###Always use Colab GPU (**VERY VERY VERY IMPORTANT!**)
You need to use a Colab GPU so the Voice Changer can work faster and better\
Use the menu above and click on **Runtime** » **Change runtime** » **Hardware acceleration** to select a GPU (**T4 is the free one**)

---

<br>

# **Credits and Support**
Realtime Voice Changer by [w-okada](https://github.com/w-okada)\
Colab files updated by [rafacasari](https://github.com/Rafacasari)\
Recommended settings by [YunaOneeChan](https://github.com/YunaOneeChan)

Need help? [AI Hub Discord](https://discord.gg/aihub) » ***#help-realtime-vc***

---

In [ ]:
# @title **[Optional]** Connect to Google Drive
# @markdown Using Google Drive can improve load times a bit and your models will be stored, so you don't need to re-upload every time that you use.
import os
from google.colab import drive

if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

%cd /content/drive/MyDrive

In [ ]:
# @title **[1]** Clone repository and install dependencies
# @markdown This first step will download the latest version of Voice Changer and install the dependencies. **It will take around 2 minutes to complete.**

!git clone --depth 1 https://github.com/w-okada/voice-changer.git &> /dev/null

%cd voice-changer/server/
print("\033[92mSuccessfully cloned the repository")

!apt-get install libportaudio2 &> /dev/null
!pip install pyworld!pip install onnxruntime-gpu uvicorn faiss-gpu fairseq jedi google-colab moviepy decorator==4.4.2 sounddevice numpy==1.23.5 pyngrok --quiet
!pip install -r requirements.txt --no-build-isolation --quiet
# Maybe install Tensor packages?
#!pip install torch-tensorrt
#!pip install TensorRT
print("\033[92mSuccessfully installed all packages!")

In [ ]:
# @title **[2]** Start Server **using ngrok** (Recommended | **need a ngrok account**)
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a while (~1-2 minutes)

# @markdown ---
# @markdown You'll need a ngrok account, but **it's free**!
# @markdown ---
# @markdown **1** - Create a **free** account at [ngrok](https://dashboard.ngrok.com/signup)\
# @markdown **2** - If you didn't logged in with Google or Github, you will need to **verify your e-mail**!\
# @markdown **3** - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, copy it and place it here:
from pyngrok import conf, ngrok

Token = '' # @param {type:"string"}
# @markdown **4** - Still need further tests, but maybe region can help a bit on latency?\
# @markdown `Default Region: us - United States (Ohio)`
Region = "us - United States (Ohio)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

MyConfig = conf.PyngrokConfig()

MyConfig.auth_token = Token
MyConfig.region = Region[0:2]

conf.get_default().authtoken = Token
conf.get_default().region = Region[0:2]

conf.set_default(MyConfig);

# @markdown ---
# @markdown If you want to automatically clear the output when the server loads, check this option.
Clear_Output = True # @param {type:"boolean"}

import portpicker, subprocess, threading, time, socket, urllib.request
PORT = portpicker.pick_unused_port()

from IPython.display import clear_output, Javascript

from pyngrok import ngrok
ngrokConnection = ngrok.connect(PORT)
public_url = ngrokConnection.public_url

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  clear_output()
  print("------- SERVER READY! -------")
  print("Your server is available at:")
  print(public_url)
  print("-----------------------------")
  display(Javascript('window.open("{url}", \'_blank\');'.format(url=public_url)))

threading.Thread(target=iframe_thread, daemon=True, args=(PORT,)).start()

!python3 MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json

In [ ]:
# @title **[Optional]** Start Server **using localtunnel** (ngrok alternative | no account needed)
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a while (~1-2 minutes)

# @markdown ---
!npm config set update-notifier false
!npm install -g localtunnel
print("\033[92mLocalTunnel installed!")
# @markdown If you want to automatically clear the output when the server loads, check this option.
Clear_Output = True # @param {type:"boolean"}

import portpicker, subprocess, threading, time, socket, urllib.request
PORT = portpicker.pick_unused_port()

from IPython.display import clear_output, Javascript

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  clear_output()
  print("Use the following endpoint to connect to localtunnel:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')

threading.Thread(target=iframe_thread, daemon=True, args=(PORT,)).start()


!python3 MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json \
  --colab True

# In Development | **Need contributors**

In [ ]:
# @title **[BROKEN]** Start Server using Colab Tunnels (trying to fix this TwT)
# @markdown **Issue:** Everything starts correctly, but when you try to use the client, you'll see in your browser console a bunch of errors **(Error 500 - Not Allowed.)**

import portpicker, subprocess, threading, time, socket, urllib.request
PORT = portpicker.pick_unused_port()

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab.output import serve_kernel_port_as_window
  serve_kernel_port_as_window(PORT)

threading.Thread(target=iframe_thread, daemon=True, args=(PORT,)).start()

!python3 MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json